### Linear Regression
4 Aussmptions: Linearity; Independence of observations; Homoscedasticity (var(y) constant for x); Normality (fixed x, y is normal)

Minimize "Mean Squred Error" to estimate parameters or called "Ordinary Least Squares" 

OLS has five assumptions: Linearity(in parameters); Random sampling of obs; Zero mean of error|x; No multicollinearity; Homoscedasticity and no autocorrelation among errors

Implementation: key logic - initialize parameters; compute parameters; update gradient; helper funtion for details

In [1]:
# main
def linear_regression(x,
                     y,
                     iterations = 100,
                     learning_rate = 0.005):
    n, p = len(x), len(x[0])
    beta_0, beta_other = initialize_parameter(p)
    for _ in range(iterations):
        gradient_beta_0, gradient_beta_other = compute_gradient(x,
                                                               y,
                                                               beta_0,
                                                               beta_other,
                                                               n,
                                                               p)
        beta_0, beta_other = update_parameter(beta_0,
                                             beta_other,
                                             learning_rate,
                                             gradient_beta_0,
                                             gradient_beta_other)
    return beta_0, beta_other

In [ ]:
def initialize_parameter(dim):
    beta_0 = 0
    beta_other = [random.random() for _ in range(dim)]
    return beta_0, beta_other

def compute_gradient(x,y,beta_0,beta_other,n,dim):
    gradient_beta_0 = 0 
    gradient_beta_other = [0] * dim
    
    for i in range(n):
        y_i_hat = sum(x[i][j]*beta_other[j] for j in range(dim)) + beta_0
        derror_dy_i = -2*(y[i] - y_i_hat)
        gradient_beta_0 += derror_dy_i / n
        for j in range(p):
            gradient_beta_other[j] += derror_dy * x[i][j] / n
    return gradient_beta_0, gradient_beta_other

def update_parameter(beta_0, beta_other, gradient_beta_0, gradient_beta_other, learning_rate):
    beta_0 += learning_rate * gradient_beta_0
    for j in range(len(beta_other)):
        beta_other[j] += learning_rate * beta_other[j]
    return beta_0, beta_other    

### Logistic Regression
Binary classification problem (compute probability & classify w/ threshold)
MLE -> Loss function = -1/n*ln(MLE)

In [ ]:
def logistic_regression(x,y,iterations=100,learning_rate=0.05):
    n,p = len(x), len(x[0])
    beta_0, beta_other = initialize_parameter(p)
    
    for _ in range(len(iterations)):
        gradient_beta_0, gradient_beta_other = compute_gradient(x,
                                                                y,
                                                                beta_0,
                                                                beta_other,
                                                                n,
                                                                p)
        beta_0, beta_other = update_parameters(beta_0,
                                              beta_other,
                                              gradient_beta_0,
                                              gradient_beta_other,
                                              learning_rate) # add mini_batch size if using mini batch gradient descent
    return beta_0, beta_other     

In [ ]:
def initialize_parameter(dim):
    beta_0 = 0
    beta_other = [random.random() for i in range(dim)]
    return beta_0, beta_other

def sigmoid(x,beta_0,beta_other):
    return 1 / (1+np.exp(-(x.dot(beta_other)+beta_0)))

def compute_gradient(x,y,beta_0,beta_other,n,p):
    # 1. initialize gradients
    gradient_beta_0 = 0
    gradient_beta_other = [0] * p
    
    # 2. loop through the samples to add up the gradient at each sample
    for i,point in enumerate(x):
        pred = sigmoid(point,beta_0,beta_other)
        gradient_beta_0 += (y[i]-pred) / n
        for j,feature in enumerate(point):
            gradient_beta_other[j] += (y[i]-pred) / n * feature
    # 3. return the gradients
    return gradient_beta_0, gradient_beta_other

def update_parameters(beta_0,beta_other,gradient_beta_0,gradient_beta_other,learning_rate):
    beta_0 += gradient_beta_0 * learning_rate
    for j in range(len(beta_other)):
        beta_other[j] += gradient_beta_other[j] * learning_rate
    return beta_0, beta_other 

def compute_gradient_mini_batch(x,y,beta_0,beta_other,n,p,batch_size):
    gradient_beta_0 = 0
    gradient_beta_other = [0] * p
    
    for _ in range(batch_size):
        i = random.randint(0,n-1)
        point = x[i]
        pred = sigmoid(point,beta_0,beta_other)
        gradient_beta_0 += (pred-y[i]) / n
        for j,feature in enumerate(beta_0):
            gradient_beta_other[j] += (pred-y[i]) / n * point[i][j]
    return beta_0, beta_other

### K Nearest Neighbors
Non-parameteric method; one parameter k and distance

In [ ]:
class KNN:
    def __init__(self):
        self.x = None
        self.y = None
    def train(self, x, y):
        self.x = x
        self.y = y
    def distanct(self, x1, x2):
        # use Euclidean distance: square root of MSE
        SSE = sum([(x1_i-x2_i)**2 for x1_i, x2_i in zip(x1,x2)])
        return sqrt(SSE)
    def predict_continuous(self, new_x, k): # regression
        distance_label = [(self.distance(new_x, train_x), train_y) for train_x, train_y in zip(self.x, self.y)]
        k_neighbors = sorted(distance_label)[:k]
        return sum(label for _, label in k_neighbors)/k
    def predict_categorical(self, new_x, k): # classification
        distance_label = [(self.distance(new_x, train_x), train_y) for train_x, train_y in zip(self.x, self.y)]
        k_neighbors = sorted(distance_label)[:k]
        k_neighbors_labels = [label for _, label in k_neighbors]
        return Counter(k_neighbors_labels).most_common()[0][0]

### K Means Clustering
* Step 1: initialize k entroids
* Step 2: calculate distance, pick class and update centroids
* Step 3: check the threshold, if smaller than threshold stop

In [ ]:
# for simplicity use two dimensional data point as example
def K_Means(data,k):
    centroids = centroids_initialize(data,k)
    while True: # until converge
        old_centroids = centroids
        labels = get_labels(data, centroids, k)
        centroids = update_centroids(data, labels)
        
        if should_stop(old_centroids, centroids):
            break
    return labels # need to return the clustering classes instead of centroids!

def random_sample(low, high):
    return low + (high - low)*random.random() # 0~1

def centroids_initialize(data,k):
    x_min, x_max = float('inf'), float('-inf')
    y_min, y_max = float('inf'), float('-inf')
    
    for point in data:
        x_min = min(x_min, point[0])
        x_max = max(x_max, point[0])
        y_min = min(y_min, point[1])
        y_max = max(y_max, point[1])
    
    centroids = []
    for i in range(k):
        centroids.append([random_sample(x_min, x_max),
                         random_sample(y_min, y_max)])
    return centroids

def distance(point1, point2):
    return ( (point1[0] - point2[0])**2+(point1[1] - point2[1])**2 ) **0.5

def get_labels(data, centroids):
    
    labels = []
    for x in data:
        label = -1
        min_distance = float('inf')        
        for j,centroid in enumerate(centroids):
            distance_j = distance(x, centroid)
            if distance_j < min_distance:
                min_distance = distance_j
                label = j
        labels.append(label)
    return labels

def update_centroids(data,labels,k):
    new_centroids = [[0,0] * k] # get nuerator
    counts = [] # get denominator
    for x,label in zip(data,labels):
        new_centroids[label][0] += x[0]
        new_centroids[label][1] += x[1]
        count[label] += 1
    for j,(x,y) in enumerate(new_centroids):
        new_centroids[j] = [x/count[j], y/count[j]]
    return new_centroids
        
    

### Decision Tree
* Step 1: Create root -> requires find the best split by traversing all the attributes and all the values, which then require test_split function for each attribute at a value and gini calculation function.
* Step 2: Recursively split for childrens -> DFS pattern; use max_depth, min_child to early return; create terminal nodes
* Step 3: Return the tree
* https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [ ]:
# suppose the last column is target

def compute_gini(groups, classes):
    gini = 0.0
    n_instances = sum([len(group) for group in groups])
    # gini of each split = sum of (gini of each group * size / total sample size)
    # gini of each group = 1 - sum of (p^2 of each class)
    
    for group in groups:
        size = len(group)
        if size = 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p**2
        gini += (1 - score) * (size / n_instances)
    return gini
         
def test_split(rows, index, index_value):
    left, right = list(), list()
    for row in rows:
        if row[index] > index_value:
            right.append(row)
        else:
            left.append(row)
    return left, right

def get_split(rows):
    # best_index, best_index_value, best_groups, best_gini
    classes = list(set(row[-1] for row in rows))
    best_index, best_index_value, best_groups, best_gini = 999, 999, [], 999
    for index in range(len(rows[0])-1): # not including the target
        for row in rows:
            groups = test_split(rows, index, row[index])
            gini = compute_gini(groups, classes)
            if gini < best_gini:
                best_index, best_index_value, best_groups, best_gini = index, row[index], groups, gini
    return {'index': best_index,
           'value': best_index_value,
           'groups': best_groups}

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key = outcomes.count) # the mode class

def split(node, max_depth, min_child, depth):
    left, right = node['groups']
    del node['groups']
    # below are certain situations for terminal node, o.w. recursive
    if not left or not right: # actually no split
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
    if len(left) <=  min_child:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_child, depth + 1)
    if len(right) <=  min_child:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_child, depth + 1)    
    

In [ ]:
def build_tree(train, max_depth, min_child):
    root = get_split(train)
    split(root, max_depth, min_child, 1)
    return root

In [ ]:
def print_tree(): # did not understand well

In [ ]:
def predict(node, row):
    
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            predict(node['right'], row)
        else:
            return node['right']

### Cracking the machine learning questions
